In [ ]:
# import relevant packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
import df2gspread as d2g
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from forex_python.converter import CurrencyRates

In [ ]:
# List of cities from Europe, courtesy of ChatGPT
cities=['Rome', 'Milan', 'Naples', 'Turin', 'Palermo', 'Genoa', 'Bologna', 'Florence', 'Bari', 'Catania', 'Madrid', 'Barcelona', 'Valencia', 'Seville', 'Zaragoza', 'Málaga', 'Murcia', 'Palma', 'Las Palmas', 'Bilbao', 'Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Lisbon', 'Porto', 'Amadora', 'Braga', 'Coimbra', 'Funchal', 'Setúbal', 'Agualva-Cacém', 'Almada', 'Queluz', 'Zurich', 'Geneva', 'Basel', 'Bern', 'Lausanne', 'Winterthur', 'Lucerne', 'St. Gallen', 'Lugano', 'Biel', 'Vienna', 'Graz', 'Linz', 'Salzburg', 'Innsbruck', 'Klagenfurt', 'Villach', 'Wels', 'Sankt Pölten', 'Dornbirn', 'Berlin', 'Hamburg', 'Munich', 'Cologne', 'Frankfurt', 'Stuttgart', 'Düsseldorf', 'Dortmund', 'Essen', 'Bremen', 'Copenhagen', 'Aarhus', 'Aalborg', 'Odense', 'Esbjerg', 'Randers', 'Kolding', 'Horsens', 'Vejle', 'Roskilde', 'London', 'Birmingham', 'Leeds', 'Glasgow', 'Sheffield', 'Bradford', 'Liverpool', 'Edinburgh', 'Manchester', 'Belfast', 'Brussels', 'Antwerp', 'Ghent', 'Charleroi', 'Liege', 'Bruges', 'Namur', 'Mons', 'Aalst', 'La Louvière', 'Oslo', 'Bergen', 'Trondheim', 'Stavanger', 'Bærum', 'Ålesund', 'Tønsberg', 'Drammen', 'Sarpsborg', 'Skien', 'Stockholm', 'Gothenburg', 'Malmö', 'Uppsala', 'Sollentuna', 'Västerås', 'Örebro', 'Linköping', 'Helsingborg', 'Jönköping', 'Helsinki', 'Espoo', 'Tampere', 'Vantaa', 'Oulu', 'Turku', 'Jyväskylä', 'Lahti', 'Kuopio', 'Pori']

In [ ]:
# create dictionary of relevant cities as keys, and empty values to create the correct json structure to receive the data from Numbeo
diz=dict.fromkeys(cities)
for city in diz:
  diz[city] = ({'key1': 'value1'})

In [ ]:
# Iterate over the numbeo website with the cities from the above dictionary, and take the category and value to place in the nested json format from above
for city in diz:
  #print('https://www.numbeo.com/cost-of-living/in/'+city)
  url='https://www.numbeo.com/cost-of-living/in/'+city
  page = requests.get(url)
  #print(page.status_code)
  soup = BeautifulSoup(page.content, 'html.parser')
  table = soup.find('table', {'class': 'data_wide_table'})
  try:
    rows = table.find_all('tr')
  except:
    continue
  n=1
  for row in rows:
    # Find all cells in the row
      cells = row.find_all('td')
    # If there are cells in the row, append the data to the dictionary
      if cells:
        key=cells[0].text
        value=cells[1].text
        diz[city][key]=value

In [ ]:
# insert dictionary into a table, and perform data cleaning
raw_data=pd.DataFrame.from_dict(diz, orient='index')
raw_data.reset_index(inplace=True)
raw_data = raw_data.rename(columns = {'index':'City'})
raw_data.drop('key1', axis=1, inplace=True)
perc = 25.0
min_count =  int(((100-perc)/100)*raw_data.shape[1] + 1)
raw_data = raw_data.dropna( axis=0, 
                    thresh=min_count)
raw_data.reset_index(drop=True, inplace=True)
raw_data.rename(columns=lambda x: x.strip(), inplace=True)

In [ ]:
# transform all currencies into euros for easy comparison, by taking the data from Forex
currency=[]
for index, row in raw_data.iterrows():
  currency.append(row["Meal, Inexpensive Restaurant"])

sep = '\xa0'
currency_clean=[]
for i in range(len(currency)):
  cur=currency[i].split(sep, 1)[1]
  if cur=='€':
    currency_clean.append('EUR')
  if cur=='Fr.':
    currency_clean.append('CHF')
  if cur=='kr':
    currency_clean.append('SEK')
  if cur=='£':
    currency_clean.append('GBP')
  #currency_clean.append(currency[i].split(sep, 1)[1])

raw_data['Currency']=currency_clean

c=CurrencyRates()
chf=(c.get_rate('EUR', 'CHF'))
sek=(c.get_rate('EUR', 'SEK'))
gbp=(c.get_rate('EUR', 'GBP'))

cur_exch=[]
for i in range(len(currency_clean)):
  if currency_clean[i]=='EUR':
    cur_exch.append('1')
  if currency_clean[i]=='CHF':
    cur_exch.append(chf)
  if currency_clean[i]=='SEK':
    cur_exch.append(sek)
  if currency_clean[i]=='GBP':
    cur_exch.append(gbp)

raw_data['Currency_rates']=cur_exch

In [ ]:
num_columns = len(raw_data.columns)

for i in range(1, num_columns - 3):
  col_name= raw_data.columns[i]
  for index, row in raw_data.iterrows():
    multiplier=float(row[-1])
    value = row[col_name]
    try:
      number=(float(value.split(sep, 1)[0].strip().replace(',','')))/multiplier
    except:
      number = 0
      continue
    row[col_name]=number

In [ ]:
# load data to google sheet
sh = gc.create('Numbeo')
worksheet = gc.open('Numbeo').sheet1
set_with_dataframe(worksheet, raw_data)